# Wstęp do uczenia maszynowego - Notebook 3, version for students
**Author: Michał Ciach**  
Exercises denoted with a star \* are optional. They may be more difficult or time-consuming.  


## Description
In today's class, we'll learn a method of statistical inference called *hypothesis testing*.   

*Theory time.* The general idea of statistical hypothesis testing is to set up a statistical hypothesis - e.g., that some parameter is greater than 0 - and try to *reject it* by showing that our data would be very unlikely to observe if this hypothesis was true.     

Why do we reject statistical hypotheses rather than confirm them? Because in general, confirming them is impossible using random samples. For example, let's suppose that we study the toxicity of a new drug by giving it to 10 rats and observing them for a week. Just because the rats didn't get sick may suggest that the drug is safe, but it doesn't prove it - after all, it's just 10 animals observed for a short time. On the other hand, if all the 10 animals get sick (and the control group is fine), we can definitely reject the hypothesis that the drug is safe.   

Another example: let's suppose we have a hypothesis that the average log-length of a human protein is equal 2.30, and we sample 200 proteins to verify it. If their average log-length turns out to be similar to 2.30 - for example, 2.31 with a standard deviation 0.1 - this doesn't prove our hypothesis. The true average log-length may still be equal, for example, 2.29 or 2.305. However, if the average log-length of our sample turns out to be 2.70 with a standard deviation 0.1, this is a strong evidence that our hypothesis is false.  

More generally: just because some data somewhat agree with our beliefs doesn't prove that our beliefs are true. However, if the data contradicts our beliefs, then it proves our beliefs are false.  





## Data & library imports

In [1]:
!pip install gdown

In [2]:
!gdown https://drive.google.com/uc?id=1lSiX67iYeqYO8S5251ekN3y7TJ84RXUy
!gdown https://drive.google.com/uc?id=1UmqDDwBWfhQt5gIbp1rfw69m4FMIGx7w

Downloading...
From: https://drive.google.com/uc?id=1lSiX67iYeqYO8S5251ekN3y7TJ84RXUy
To: /content/3. citizen incomes.tsv
100% 40.6k/40.6k [00:00<00:00, 77.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1UmqDDwBWfhQt5gIbp1rfw69m4FMIGx7w
To: /content/3. protein_lengths.tsv
100% 29.3M/29.3M [00:01<00:00, 18.0MB/s]


In [3]:
!pip install --upgrade scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.2 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4


In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.stats import t as tstud
from scipy.stats import ttest_ind, ttest_rel, ttest_1samp, norm, kstest, mannwhitneyu, shapiro, chisquare, chi2_contingency
import plotly.graph_objects as go
from statsmodels.stats.multitest import fdrcorrection

In [5]:
protein_lengths = pd.read_csv('3. protein_lengths.tsv', sep='\t')
protein_lengths['LogLength'] = np.log10(protein_lengths['Protein length'])
protein_lengths

,Scientific name,Common name,Protein ID,Protein length,LogLength
0,Homo sapiens,Human,NP_000005.3,1474,3.168497
1,Homo sapiens,Human,NP_000006.2,290,2.462398
2,Homo sapiens,Human,NP_000007.1,421,2.624282
3,Homo sapiens,Human,NP_000008.1,412,2.614897
4,Homo sapiens,Human,NP_000009.1,655,2.816241
...,...,...,...,...,...
648731,Imleria badia,Bay bolete (mushroom),KAF8560453.1,494,2.693727
648732,Imleria badia,Bay bolete (mushroom),KAF8560454.1,737,2.867467
648733,Imleria badia,Bay bolete (mushroom),KAF8560455.1,554,2.743510
648734,Imleria badia,Bay bolete (mushroom),KAF8560456.1,813,2.910091


In [6]:
human_protein_lengths = protein_lengths.loc[protein_lengths['Common name'] == 'Human'].copy()
# Note: without .copy(), some versions of Pandas may return a View.
# This may interfere with adding a new column to human_protein_lengths.
print(human_protein_lengths.head())
print()
print(human_protein_lengths.describe())

  Scientific name Common name   Protein ID  Protein length  LogLength
0    Homo sapiens       Human  NP_000005.3            1474   3.168497
1    Homo sapiens       Human  NP_000006.2             290   2.462398
2    Homo sapiens       Human  NP_000007.1             421   2.624282
3    Homo sapiens       Human  NP_000008.1             412   2.614897
4    Homo sapiens       Human  NP_000009.1             655   2.816241

       Protein length      LogLength
count   136193.000000  136193.000000
mean       692.655775       2.711540
std        746.993628       0.329892
min         12.000000       1.079181
25%        316.000000       2.499687
50%        514.000000       2.710963
75%        842.000000       2.925312
max      35991.000000       4.556194


In [7]:
citizen_incomes = pd.read_csv('3. citizen incomes.tsv', sep='\t')
citizen_incomes

,Country,Income
0,A,43806.926920
1,A,62825.327197
2,A,46660.685387
3,A,54625.879014
4,A,41976.428035
...,...,...
1995,B,55338.284370
1996,B,95699.796316
1997,B,34417.930506
1998,B,20637.868432


## Testing the value of the mean

To illustrate the basic concepts of statistical hypothesis testing, we'll start with simple tests for a hypothesis that the true mean value is equal to some value $\mu_0$, with an alternative hypothesis that it's different:  

$$H_0: \mu = \mu_0$$
$$H_1: \mu \neq \mu_0$$  
Note the lack of hats above any of the symbols - the hypotheses are about parameters, not estimators or any other random values.   

**Exercise 1.** Consider the human protein log-length data in the `human_protein_lengths` data frame. Consider the following two null hypotheses: $H_0^{(1)}: \mu = 2.711540$ and $H_0^{(2)}: \mu = 6$. We'll use the Student's t test to verify the hypotheses using a random sample.  

1. Select a random sample of protein log-lengths of size $N=20$.   
2. Calculate the test statistics for one-sample Student's t-tests with the assumption that the standard deviation is unknown and is estimated from the sample. Pay attention which kind of the variance estimator you need to use (biased or unbiased). How many test statistics do you need to calculate to test the two hypotheses, $H_0^{(1)}$ and $H_0^{(2)}$?   
3. Use the `tstud.ppf` (i.e. the quantile function of Student's t distribution) to calculate the critical set on the significance level 5% (i.e., Type I error risk 5%). Pay attention to the shape of the critical set - for our alternative hypothesis $H_1$, this set is a union of two semi-lines. How many quantile values do you need to calculate to test the two hypotheses, $H_0^{(1)}$ and $H_0^{(2)}$?   
  3.1. Based on the values of the test statistic and the critical set, do we reject our null hypotheses? Did we correctly detect which hypothesis is true and which is false?    
4. Use the `tstud.cdf` to calculate the p-values. Again, pay attention to the shape of the critical set. How many cdf values do you need to calculate to test the two hypotheses, $H_0^{(1)}$ and $H_0^{(2)}$?  
  4.1. Based on the p-values, do we reject any of our hypotheses on the significance level 5%? Did we correctly detect which hypothesis is true and which is false?     
5. Compare your results to the Student's t test implementation in `scipy`. The appropriate test has already been loaded in the *Data & imports* section.   
6. Are there any assumptions of the test that are violated? If so, how strongly and what effect could it have on the test result?  
7. Based on the results of this exercise, can you conclude that $N=20$ is enough to prove that $\mu=2.711540$? Does the answer to this question depend on $N$?   

In [ ]:
N = 20
mu_1 = 2.711540
mu_2 = 6
log_sample = human_protein_lengths['LogLength'].sample(N)
mean_sample = np.mean(log_sample)
std_sample = np.std(log_sample, ddof=1)
t1 = (mean_sample - mu_1) / std_sample * np.sqrt(N)
t2 = (mean_sample - mu_2) / std_sample * np.sqrt(N)
print("Estimated mean:", mean_sample)
print("Value of the test statistic for H^1_0:", t1)
print("Value of the test statistic for H^2_0:", t2)

alfa = 0.05
ci = tstud.ppf([alfa/2, 1-alfa/2], df=N-1)
print(f"We reject the hypothesis if T < {ci[0]} or T > {ci[1]}")
print("P-value for H^1_0:", tstud.cdf(t1, df=N-1))
print("P-value for H^2_0:", tstud.cdf(t2, df=N-1))

print("We do not reject H^1_0 because its p-value is above the 0.05")
print("We reject H^2_0 because its p-value is below the 0.05")

Estimated mean: 2.762522910582695
Value of the test statistic for H^1_0: 0.5731799142508976
Value of the test statistic for H^2_0: -36.39762460190544
We reject the hypothesis if T < -2.0930240544082634 or T > 2.093024054408263
P-value for H^1_0: 0.7133775451747701
P-value for H^2_0: 2.441391182375396e-19
We do not reject H^1_0 because its p-value is above the 0.05
We reject H^2_0 because its p-value is below the 0.05


**Exercise 2.** In this exercise, we'll see a more useful application of statistical hypothesis testing: comparing two populations. Say we want to use a random sample to check if the log-lengths of human proteins are, on average, higher than the ones of another organism - like the bay bolete (a kind of mushroom).

1. What are the appropriate null and alternative hypotheses if we want to use a random sample to show that human proteins are longer in terms of the average log-length?     
2. Select a random sample of human proteins ($N_\text{human}=20$) and a random sample of bay bolete proteins ($N_\text{bolete}=20$) from the `protein_lengths` data frame.
3. Use the two-sample Student's t-test implemented in the `ttest_ind` function from the `scipy` library to calculate the p-value. Pay attention to the `alternative` keyword, as well as to any keyword arguments that may correspond to assumptions such as the equality of variances.   
4. Based on the p-value, do we reject $H_0$ on a significance level 5%? Did we confirm that humans have longer or shorter proteins than the mushroom (in terms of the log-length)?      
5.\*\* Implement your own test and compare the p-value. You can use the equations described [here](https://en.wikipedia.org/wiki/Welch%27s_t-test).  
6. What happens if we take a reverse hypothesis - that humans have lower protein log-lengths than the mushroom? Did our results confirm anything now?  
  6.1. Is the result of the test true? Compare the true average log-lengths of the two organisms.  
  6.2. Is the result of the test from point 5 true?   
7.\* Do a test for a null hypothesis that the average protein log-lengths are equal, and an alternative that they are different (in any direction; a so-called *two-sided* alternative hypothesis). Explain the difference in the results compared to the previous points.    
  7.1. Roughly speaking, what is the difference in the shape of the critical region for a one-sided and a two-sided alternative hypothesis?   
  7.2.\* Calculate the ratio of the p-value for the two-sided alternative to the p-value for the one-sided alternative that the human log-lengths are smaller than the bolete log-lengths. Explain the result.  
8. Can we use the Student's t test to test protein lengths rather than log-lengths?   


Null Hypothesis (H0): The average log-length of human proteins is equal to or less than the average log-length of bay bolete proteins.


Alternative Hypothesis (H1): The average log-length of human proteins is greater than the average log-length of bay bolete proteins.

In [ ]:
print()
N = 20
human = protein_lengths[protein_lengths['Common name'] == 'Human']['LogLength'].sample(N)
bolete = protein_lengths[protein_lengths['Common name'] == 'Bay bolete (mushroom)']['LogLength'].sample(N)
res = ttest_ind(human, bolete, alternative='greater', equal_var=False)
print("Scipy results for Human > Bolete alternative:", res)
res = ttest_ind(human, bolete, alternative='less', equal_var=False)
print("Scipy results for Human < Bolete alternative:", res)


Scipy results for Human > Bolete alternative: TtestResult(statistic=3.2452381287140337, pvalue=0.0012326823220385685, df=37.64835731955117)
Scipy results for Human < Bolete alternative: TtestResult(statistic=3.2452381287140337, pvalue=0.9987673176779615, df=37.64835731955117)


In [ ]:
# put your code here


Scipy results for Human > Bolete alternative: TtestResult(statistic=2.7800057340686686, pvalue=0.9956948655237339, df=35.78441097644484)
Scipy results for Human < Bolete alternative: TtestResult(statistic=2.7800057340686686, pvalue=0.0043051344762661614, df=35.78441097644484)
Scipy results for a two-sided alternative: TtestResult(statistic=2.7800057340686686, pvalue=0.008610268952532323, df=35.78441097644484)
Two-sided p-value divided by the one-sided p-value: 2.0
Manually calculated value of the T statistic: 2.7800057340686686
Manually calculated p-value (one-sided): 0.004305134476266126
                       Protein length  LogLength
Common name                                     
Apple                      464.063053   2.573145
Bay bolete (mushroom)      378.286030   2.443671
Chicken                    733.153531   2.732487
Chimpanzee                 742.965914   2.727545
European goshawk           740.431115   2.740182
Gorilla                    691.720583   2.707579
Human       

## Non-parametric tests

The two-sample Student's t-test assumes a normal distribution of the populations. When this assumption is only slightly violated, like for the protein log-length data, the results may still be reliable, especially for large sample sizes. For many data sets, as the sample size increases, the distribution of the estimator of the mean converges to the Normal one (this is guaranteed by the Central Limit Theorem). This means that the estimator of the mean is distributed "more normally" than the original data (where "more normal" refers to the distance between the cumulative distribution functions). This increases the robustness of the t-test for small deviations from normality. However, when this assumption is strongly violated, like for the non-transformed protein length data, the results are no longer reliable. One way to solve this problem is to use non-parametric tests. A non-parametric test is defined as a test that does not rely on the assumption of a distribution of the data.   

One of the most common non-parametric tests is the Mann-Whitney U-test, also known as the two-sample Wilcoxon's test. It's often used as a replacement for the Student's t-test when the data is not distributed normally. However, the null hypotheses of these two tests are different, and it's important to understand this difference to avoid misleading results.

In contrast to the Student's t-test, the Mann-Whitney's one doesn't test the equality of parameters like the mean - hence the name *non-parametric*. Instead, its null hypothesis is that $\mathbb{P}(X > Y) = 1/2$, i.e., that if we take a random observation $X$ from the first sample, and a random observation $Y$ from the second sample, it's equally likely that the first is greater or smaller than the second. A one-sided alternative hypothesis may be, e.g., that  $\mathbb{P}(X > Y) > 1/2$, i.e., that samples from the first population tend to be larger than sample from the second one. In this case, we say that the first sample is *stochastically greater* than the second one.  

Sidenote: the actual null hypothesis of the Mann-Whitney test is slightly different, but the one described above is a very close approximation that's much simpler to interpret and use in practice.   



**Exercise 3.\*** Implement you own version of the Mann-Whitney's test. You can find the necessary equations [here](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test) (use the normal approximation for the test statistic).

1. Use your implementation to test whether the protein lengths are higher in human than in the bay bolete (use a random sample of size $N$ of your choice).
2. Compare your results to the `mannwhitneyu` function from `scipy`. Pay attention to the default parameters to obtain identical results.  
3. Compare the results of the Mann-Whitney's test to the Student's t  test. Are the results of the two tests consistent? Can you conclude that one of the organisms has longer proteins? Is this a correct result?  


In [ ]:
N = 21
human = protein_lengths[protein_lengths['Common name'] == 'Human']['LogLength'].sample(N)
bolete = protein_lengths[protein_lengths['Common name'] == 'Bay bolete (mushroom)']['LogLength'].sample(N)

print(mannwhitneyu(human, bolete, alternative='greater'))

MannwhitneyuResult(statistic=342.0, pvalue=0.0011674002897815873)


In [ ]:
# put your code here


Scipy test results for lengths:
TtestResult(statistic=2.8755078122880064, pvalue=0.00333359425357163, df=36.826621370440954)
MannwhitneyuResult(statistic=323.0, pvalue=0.0004604566971440039)

Manual test results for lengths:
Manually computed value of the U test statistic: 323.0
Expected value of U: 200.0 with standard deviation: 36.968455021364726
Z-score: 3.327160951922825
Manually computed p-value: 0.000438678411438298
MannwhitneyuResult(statistic=323.0, pvalue=0.00043867841143833047)


**Exercise 4.** The `citizen_income` data frame, loaded in the *Data & modules* section, contains the information about the yearly income in USD of randomly sampled individuals from two countries, encoded as Country `A` and Country `B`. Use an appropriate statistical test to check whether citizens of one of the countries earn more than citizens of the other country. If you use more than one test and get contradictory results, explain why that happens.   

In [ ]:
# put your code here


T-test results: TtestResult(statistic=-2.081134180586558, pvalue=0.01883558596169213, df=1017.7689156871639)
U-test results: MannwhitneyuResult(statistic=670587.0, pvalue=3.827400955390909e-40)
Average incomes:                Income
Country              
A        54555.100828
B        60575.000000
Median incomes:                Income
Country              
A        54654.283542
B        30336.007047


**Exercise 5.\*\*** In this exercise, we'll see how the violations of test assumptions influence the distribution of the test statistics under the null hypothesis.   

1. Formulate a statistical hypothesis about the incomes of citizens of country `B` which you can test using any version of the Student's t test, and in which the null hypothesis $H_0$ is *true*. Note that in this exercise we allow hypothesis that violates assumptions of the test.   
  1.1.\* Optionally, formulate a hypothesis which you can test using both Student's t and Mann-Whitney's u test.  
2. Which assumptions of your test are violated on this data set? Are they approximately satisfied for large sample sizes?    
3. Repeat the following $R=5000$ times:   
  3.1. Draw two random samples, each of size $N=10$.   
  3.2. Calculate the values of the Student's T statistic, either manually or using functions from `scipy`.   
  3.3.\* Calculate the Mann-Whitney's U statistic using functions from `scipy`.
  3.4. Save the values of the statistics in lists.  
4. Create a histogram that depicts the distribution of the statistic. Is the distribution correct (i.e. do they agree with the theoretical asumptions of the tests)? If not, how does it influence the test results?   
  4.1.\* Draw the probability density function of the theoretical distribution of the test statistics (under the null hypothesis) on the histograms.  
  4.2. Calculate the probability that your test makes a false positive error in this data set (i.e. that it incorrectly rejects a true null hypothesis; i.e. that the test statistic is within the theoretically calculated critical region).  
5. Did we analyze all the possible ways in which violated assumptions can influence test results? If not, what other problems or errors can be caused by the violated assumptions?   
6. What happens if you use protein log-lengths instead of citizen incomes?     



In [ ]:
# put your code here


Probability of Type I error for Mann-Whitney: 0.059
Probability of Type I error for Student: 0.0392


## Testing the distribution of the data

So far, we've covered testing the values of parameters and comparing two populations. Another important class are the *goodness-of-fit* tests, which are used to check if a sample can be assumed to come from some theoretical distribution, or to check if two variables have the same distributions. Probably the most commonly used tests are:  
* Chi-square test, designed for categorical data
* Kolmogorov-Smirnov test, designed for continuous data
* Shapiro-Wilk test, designed specifically for the Normal distribution

**Exercise 6.** In this exercise, we will use the Shapiro-Wilk test to try and detect whether some distributions from the previous exercises are not normal.

1. Use the Shapiro-Wilk test (`scipy.shapiro`) to test whether the distributions of citizen incomes from Exercise 4 are distributed normally (for each country separately). Can we conclude that the income in country B is distributed normally? How about country A?
2.\*\* Use the Shapiro-Wilk test to check the distribution of the test statistics from Exercise 5. Can we conclude that they are normally distributed? Does the answer depend on the value of parameters $R$ (the number of samples) and $N$ (the sample size)? If so, how?



In [ ]:
# put your code here


Testing the distribution of citizen incomes:
Country A: ShapiroResult(statistic=0.9987704777213762, pvalue=0.7355847922419261)
Country B: ShapiroResult(statistic=0.5760313435167443, pvalue=7.3099920695338e-44)

Testing the distribution of statistics from Exercise 6:
Test result for Student's T statistic: ShapiroResult(statistic=0.9948056325905207, pvalue=1.9825988394300593e-12)
Test result for Mann-Whitney's U statistic: ShapiroResult(statistic=0.9991477739009211, pvalue=0.014051880866807481)


**Exercise 7**. A bakery wants to introduce a new type of bread to its production line. It has developed three potential recipes and asked a sample of 120 people to pick their favorite one. Recipe 1 was picked by 30, Recipe 2 by 50 and Recipe 3 by 40 people. Does this mean that recipes will differ in popularity? Use a Chi-square test of goodness of fit (`scipy.chisquare`) to check this.  

In [ ]:
# put your code here


Power_divergenceResult(statistic=5.0, pvalue=0.0820849986238988)

**Exercise 8.\*\*** In this exercise, we'll use a goodness-of-fit test to crack the following message encoded with the [Ceasar cipher](https://en.wikipedia.org/wiki/Caesar_cipher):

*AOLJHLZHYJPWOLYPZVULVMAOLLHYSPLZARUVDUHUKZPTWSLZAJPWOLY
ZPAPZHAFWLVMZBIZAPABAPVUJPWOLYPUDOPJOLHJOSLAALYPUAOLWSH
PUALEAPZZOPMALKHJLYAHPUUBTILYVMWSHJLZKVDUAOLHSWOHILA*

Let $X_i$ be the frequency of the $i$-th letter of the alphabet in the above message. Use an appropriate goodness-of-fit test to find a shift of the alphabet for which the distribution of $X_i$ is statistically indistinguishable from the distribution of the letters in the English language. The frequency of each letter in the English language, and a function to shift the alphabet by a given number of letters, are given in the cell below.    




In [ ]:
letter_frequency = {
    'E': 0.1202,
    'T': 0.091,
    'A': 0.0812,
    'O': 0.0768,
    'I': 0.0731,
    'N': 0.0695,
    'S': 0.0628,
    'R': 0.0602,
    'H': 0.0592,
    'D': 0.0432,
    'L': 0.0398,
    'U': 0.0288,
    'C': 0.0271,
    'M': 0.0261,
    'F': 0.023,
    'Y': 0.0211,
    'W': 0.0209,
    'G': 0.0203,
    'P': 0.0182,
    'B': 0.0149,
    'V': 0.0111,
    'K': 0.0069,
    'X': 0.0017,
    'Q': 0.0011,
    'J': 0.0011,
    'Z': 0.0007
}

def shift_string(s, n):
  """
  Returns a string obtained from the input string s
  by shifting each letter n places down the alphabet
  """
  return ''.join(map(chr, (65 + (o-65+n)%26 for o in map(ord, s))))

In [ ]:
# put your code here


Chi-square test p-values for different shifts: 
0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0
16 0.0
17 0.0
18 0.0
19 0.1543
20 0.0
21 0.0
22 0.0
23 0.0
24 0.0
25 0.0
Decoded message:
THECAESARCIPHERISONEOFTHEEARLIESTKNOWNANDSIMPLESTCIPHERSITISATYPEOFSUBSTITUTIONCIPHERINWHICHEACHLETTERINTHEPLAINTEXTISSHIFTEDACERTAINNUMBEROFPLACESDOWNTHEALPHABET


## Multiple hypothesis testing



When performing statistical tests, we reject the null hypothesis when the p-value is below a set threshold, traditionally 0.05. A consequence of this approach is that 5% of positive results will be false positives (Type I errors; incorrect rejections of a true null hypothesis), which may be a huge number when thousands of tests are performed in large-scale studies.  

In order to limit the number of false positives, we use *multiple testing corrections*. One of the most useful ones is the Benjamini-Hochberg correction, which controls the False Discovery Rate (FDR), i.e. the proportion of false positives among all positive results. Note the difference from significance levels: the false discovery rate is the proportion of false positives among all positives in the *results* of the tests, while the significance level is the proportion of false positives among the *true negatives* (cases when $H_0$ is true).  

The assumptions and details of the Benjamini-Hochberg correction (and other common corrections) were discussed in the lecture; you may want to review them now. One of the most important assumptions is that the p-values come from a set of independent tests.  

A common way to perform the Benjamini-Hochberg correction is to transform the p-values $p_1, \dots, p_m$ in a way to obtain so-called *q-values* $q_1, \dots, q_m$, such that we have an FDR on the level of $Q$ when we accept all hypotheses $H_i$ with $q_i \leq Q$.


Do not confuse the False Discovery Rate (FDR) with other similarly named metrics, like the False Positive Rate (FPR)! FDR is the ratio of the number of false positives to the number of all reported positives, while FPR is the ratio of the number of false positives to the number of true null hypotheses. You can see a list of other common metrics [here](https://en.wikipedia.org/wiki/Confusion_matrix).   

In the next exercise, we will use *stochastic simulations* to generate our own data. First, however, let's talk about what it means to simulate a random variable.  

*Theory time.* The term *simulating a random variable* is confusing for many students, especially the ones that paid attention in probability classes. Recall that a random variable $X$ is a function from the space of elementary outcomes $\Omega$ to the space of real (or integer) numbers. When some students hear "let's simulate a sequence of random variables $X_1, \dots, X_n$, they wonder: how do we simulate a sequence of functions? Well, this is not what we mean at all.   

We define random variables as functions because we want to have a mathematical model of randomness. This is a convenient way to rigorously define what it means that a variable $X$ takes some value with some probability. For example, we can use it to formally define $\mathbb{P}(X=1) = \mathbb{P}(\{ \omega: X(\omega)  =1 \})$. We need this definition, because the concept of probability is defined on the space $\Omega$, not on the real numbers. However, this is not how we interpret or use random variables in practice.

Recall that the number of dots on a set of dice is a sequence of random variables $X_1, \dots, X_n$ before you throw the dice. When you throw them, you get realizations of the random variables, $X_1(\omega), \dots, X_n(\omega)$, which are a sequence of numbers (and often called *random numbers*). Measuring the outcome of a random experiment corresponds to fixing $\omega$ in a sequence of random variables. This is why we can interpret random variables as numerical variables. Defining them as functions is necessary only because otherwise terms like $\mathbb{P}(X=1)$ don't have any mathematical meaning.

"Fixing $\omega$" is also what we mean by simulating a sequence of random variables: we want to get a sequence of numbers $x_1, \dots, x_n$ that are realizations of a sequence of random variables $X_1, \dots X_n$, so that $x_i = X_i(\omega)$. However, simulating doesn't mean that we pick $\omega$ by hand. Remember that $\omega$ is mostly a mathematical model, not a physical being. Instead of selecting it by hand, we rely on random number generators that "fix $\omega$" for us.   

You have already encountered a few random number generators. A set of dice is one of them. When you throw the dice, you can imagine that the action of throwing corresponds to fixing $\omega$ in a sequence of random variables - and you get a sequence of numbers as a result. You probably have also used pseudo-random number generators in your programs. These are algorithms that return pseudo-random numbers between 0 and 1. When you run this algorithm, you can imagine that this corresponds to fixing $\omega$ in a random variable $X\sim U([0, 1])$ - and you get a random number as a result.   

These two random number generators (the dice and the algorithm) return numbers with very simple distributions. However, we can simulate random numbers with many other distributions, such as the Normal one. There are many techniques to do this, which usually rely on transforming random numbers distributed uniformly between 0 and 1 (formally speaking, numbers which are a realization of a sequence of random variables $X_i$ such that $X_i \sim_{iid} U([0, 1])$) (iid means independent and identically distributed, próba prosta in polish). In this course, you don't need to worry about these techniques: we will simulate random variables using algorithms which are already implemented in Python libraries.



**Exercise 9.** In this exercise, we'll learn how to use the Benjamini-Hochberg correction to control the False Discovery Rate, as well as how to simulate our own data sets. We will generate a set of samples from two different distributions and test for the value of their means, resulting in a set of tests in which some null hypotheses are true, and some are false.  

1. Generate $M=1000$ random samples of size $N=10$ from the standard normal distribution, and $M=1000$ samples of size $N=10$ from the normal distribution with expected value $\mu=1$. Use the `norm.rvs` function from `scipy` (if you need to, you can look for its documentation online).
2. For each sample, do a one-sample Student's t test to check whether the mean is non-zero. Save the p-values.      
  2.1. How many null hypotheses are true, and how many are false?   
3. Reject the null hypotheses on the significance level 5% (i.e. when the p-values are less than 0.05).   
  3.1. How many positive results (i.e. rejections of the null hypothesis) did you get?  
  3.2. How many true positives did you get?  
  3.3. How many false positives did you get?  
  3.4. How many false positives did you expect given the significance level?   
  3.5. Calculate the obtained False Positive Rate. Is it close to the significance level? Why/why not?       
  3.6. Calculate the obtained False Discovery Rate.     
  3.7. Which of the previous points depend on $N$? Which do depend on $M$? Give a theoretical argument.   
4. Perform the Benjamini-Hochberg correction on your p-values using the `fdrcorrection` function from the `statsmodels` library. Reject the null hypothesis at the FDR level 10%.  
  4.1. How many positive results did you get?  
  4.2. How many true positives did you get?  
  4.3. How many false positives did you get?  
  4.4. Calculate the obtained False Positive Rate. Is it close to the significance level? Why/why not?     
  4.5. Calculate the obtained False Discovery Rate. Is it close to the assumed 10% FDR? Why/why not?  
  4.6. How many false positives did you expect given the FDR level?  
5. Suppose all the samples were drawn from the standard normal distribution. What would be the effect of applying the Benjamini-Hochberg correction? How many false positives would you expect to obtain?    
6. \* Based on the description of the Benajmini-Hochberg procedure from [this Wikipedia article](https://en.wikipedia.org/wiki/False_discovery_rate#Benjamini%E2%80%93Hochberg_procedure), figure out how to compute the q-values given a list of p-values. Write a function that accepts a vector of p-values and returns the corresponding q-values. Compare your results to the `fdrcorrection` function from the `statsmodels` library on a p-value vector (0.01, 0.1, 0.01, 0.2, 0.01, 0.1).

In [ ]:
# put your code here


My q-values: [0.02 0.12 0.02 0.2  0.02 0.12]
Statsmodels: [0.02 0.12 0.02 0.2  0.02 0.12]

Testing
Number of tests done: 1000
Number of positives: 431
Number of true positives: 409
Number of false positives: 22
"Raw" FPR: 0.044
"Raw" FDR: 0.05104408352668213
HB number of positives: 409
HB number of true positives: 392
HB number of false positives: 17
HB FPR: 0.034
HB FDR: 0.04156479217603912


**Exercise 10.\*** In the previous exercise, you may have noticed that the results vary between the runs. Each sampling of the $2M$ samples returns a different FDR. In this exercise, we'll analyze the distribution of the FDR.  
1. Repeat the calculation of the "raw" FDR (i.e. before the correction) and the "BH" FDR (after the correction) for $R=1000$. Also calculate the True Positive Rate, i.e. the fraction of correctly identified positives.  
2. Generate histograms that compare the FDR and TPR before and after the BH correction.     
  2.1. Is there any adverse effect to using the Benjamini-Hochberg correction?  
3. Calculate the average FDR after the correction. Is it close to the assumed 10% level? Why/why not?       

In [ ]:
# put your code here


**Exercise 11.** In each of the research experiments below, there is either some procedure applied incorrectly, or it is correct but can be improved. For each point, say whether all the assumptions are satisfied, and whether the methodology can be improved (e.g. by using a different statistical test or correction procedure). If the assumptions are violated, discuss the possible effect of this kind of violation. For extra points, support your claims with theoretical arguments or simulations.        
1. A researcher studies the effect of glucose consumption on rat well-being. The researcher has partitioned $N=20$ rats into two groups: an experimental one with excess glucose in the diet, and a control one with a normal diet. After a week, the researcher has measured the weight of the rats, the length of sleep, the blood glucose level, the abdominal circumference, the body fat percentage, and the anxiety level (using the elevated plus maze assay). The researcher has compared each factor between the experimental and the control group using the two-sample Student's t test for unpaired observations, previously verifying that the compared variables are normally distributed using the Shapiro-Wilk's test. Next, the researcher has corrected the Student's t test p-values for multiple testing using the Benjamini-Hochberg correction at FDR level 20%.   
2. A researcher studies the effect of consumption of small amounts of vinegar on the increase of the blood glucose level (BGC) in mice. The researcher has partitioned $N=20$ mice into equally-sized experimental and control groups. At the beginning of the experiment, he has measured the BGC of all mice. Next, for four days, mice from both groups were given a diet enriched in glucose. Mice from the experimental groups were also given small amounts of vinegar in their diet. After four days, the BGC of all mice was measured again. Next, in each group, the researcher has tested for the increase in BGC using a two-sample Student's t-test for unpaired observations.  
3. A researcher studies the effect of artificial sweeteners on the blood glucose level (BGC) in mice. The researcher has partitioned $N=100$ mice into ten equally-sized groups: nine experimental and one control. Mice in the control group were given a normal diet. In the respective experimental groups, the mice were given food enriched in glucose; sucralose; aspartam; erythritol; stevia; xylitol; saccharine; acesulfam K; manninol. After four days, the researcher has measured the BGC of all mice. Next, the researcher has used the two-sample Student's t test for unpaired observations to compare the average BGC between the ten groups. To correct for multiple tests, the researcher has used the Benjamini-Hochberg procedure at FDR level 20%.      
4. A researcher studies the effect of salt consumption on the abdominal circumference (AC) in mice. The researcher has partitioned $N=20$ mice into equally-sized experimental and control groups. The AC of all mice was measured at the beginning of the experiment. Mice from the experimental group were given a diet enriched in salt. After a week, the AC was measured again, and the researcher used a two-sample Student's t test for paired observations in each group. The results of the test in each group were insigificant, and the researcher concluded that salt consumption does not increase the abdominal circumference in the span of one week.     

*Acknowledgement.* Exercise 8 was inspired by the following two resources: http://practicalcryptography.com/cryptanalysis/text-characterisation/chi-squared-statistic/ and https://ibmathsresources.com/2014/06/15/using-chi-squared-to-crack-codes/.  
Exercise 11 was partly inspired by the following resource: https://stats.libretexts.org/Bookshelves/Applied_Statistics/Biological_Statistics_(McDonald)/06%3A_Multiple_Tests/6.01%3A_Multiple_Comparisons

<center><img src='https://drive.google.com/uc?export=view&id=12CrUdXDAiltLBT26sG7HZ_HciIhvGyT8'></center>|